# LLM Results

In [1]:
import sys
sys.path.append("..")
from src.Eval import compute_results_jsonl, compute_results_txt
from src.Utils import get_true_pred_entities, read_IOB_file


# LLama-3 Zero-Shot

In [2]:
data_llm_zeroshot = compute_results_jsonl("../data/dataset/shs100k2/complete/test.bio", "../output/llama3_simple_0shot.jsonl")

100%|██████████| 8084/8084 [00:20<00:00, 386.39it/s]


Retag IOBs...


100%|██████████| 8084/8084 [00:18<00:00, 447.63it/s]
2024-06-20 11:16:39 root INFO: Imported 8064 predictions for 8064 true examples


INFO: 20 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.706611  0.752688  0.728922      25833         1509          0      6979        9217       34321     36559
ent_type     0.743647  0.792139  0.767128      27187          155          0      6979        9217       34321     36559
exact        0.710851  0.757204  0.733296      25988         1354          0      6979        9217       34321     36559

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.582931  0.684682  0.629723      12909          735          0      5210        8501

In [3]:
from src.Utils import get_error_analysis

missing, spurious, incorrect = get_error_analysis(data_llm_zeroshot, "IOB_pred")


# LLama-3 Few-Shot
- baseline RoBERTa still better at everything!
- results for WoA are closer but still below baseline
- main problem: many artists are missed and/or spurious!
- almost 5 times "missed" and more than 6 times "spurious" for artist

In [4]:
data_llm_fewshot = compute_results_jsonl("../data/dataset/shs100k2/complete/test.bio", 
                                         "../output/llama3_simple_4shot.jsonl")


100%|██████████| 8084/8084 [00:20<00:00, 389.96it/s]


Retag IOBs...


100%|██████████| 8084/8084 [00:18<00:00, 447.90it/s]
2024-06-20 11:17:22 root INFO: Imported 8064 predictions for 8064 true examples


INFO: 20 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.713994  0.774074  0.742821      26567         1632          0      6122        9010       34321     37209
ent_type     0.751485  0.81472   0.781826      27962          237          0      6122        9010       34321     37209
exact        0.720363  0.78098   0.749448      26804         1395          0      6122        9010       34321     37209

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.589511  0.706481  0.642718      13320          831          0      4703        8444

# LLaMa-3 More Instructions, Zero-Shot

In [5]:
data_llm_fewshot = compute_results_jsonl("../data/dataset/shs100k2/complete/test.bio", 
                                         "../output/llama3_simple+featuring_0shot.jsonl")


100%|██████████| 8084/8084 [00:20<00:00, 396.01it/s]


Retag IOBs...


100%|██████████| 8084/8084 [00:18<00:00, 446.34it/s]
2024-06-20 11:18:53 root INFO: Imported 8064 predictions for 8064 true examples


INFO: 20 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.726777  0.75024   0.738323      25749         1462          0      7110        8218       34321     35429
ent_type     0.763132  0.787768  0.775254      27037          174          0      7110        8218       34321     35429
exact        0.731689  0.75531   0.743312      25923         1288          0      7110        8218       34321     35429

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.609921  0.680863  0.643443      12837          726          0      5291        7484

## Error Analysis

In [6]:
def resolve_span_dict(span_dict: dict, text_list: list) -> dict:
    return {k: ' '.join(text_list[k[0]:k[1]+1]) for (k, v) in span_dict.items()}

missed, spurious, incorrect = get_error_analysis(data_llm_fewshot, "IOB_pred")

# missed
mask = missed.apply(len) > 0


In [60]:
data_llm_fewshot.loc[19].IOB_pred

['B-Artist',
 'I-Artist',
 'I-Artist',
 'O',
 'B-WoA',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-WoA',
 'O',
 'O',
 'O']

In [7]:
import pandas as pd

data = pd.read_parquet("../data/intermediate/shs100k2_IOB.parquet")


# RoBERTa Baseline

In [9]:
data_bert = compute_results_txt("../data/dataset/shs100k2/complete/test.bio", 
                                "../baseline/music-ner-eacl2023/output/shs100k2/complete/roberta-large/predictions.txt")


2024-06-19 11:29:36 root INFO: Imported 7725 predictions for 7725 true examples


INFO: 359 non matching IOB taglists.

 Overall
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.897494  0.915266  0.906293      27501          993          0      1553        2148       30047     30642
ent_type     0.924254  0.942557  0.933316      28321          173          0      1553        2148       30047     30642
exact        0.903139  0.921024  0.911994      27674          820          0      1553        2148       30047     30642

 Artist
schema      precision    recall        f1    correct    incorrect    partial    missed    spurious    possible    actual
--------  -----------  --------  --------  ---------  -----------  ---------  --------  ----------  ----------  --------
strict       0.890492  0.915682  0.902912      15117          392          0      1000        146